**KAGGLE TEAM: keep_it_high (ranked 28)**
**ΚΩΝΣΤΑΝΤΙΝΟΣ ΒΛΑΧΑΚΗΣ 3140025
  ΧΡΗΣΤΟΣ ΣΑΛΙΑΜΠΟΥΚΟΣ 3140176
  ΙΩΑΝΝΗΣ ΔΗΜΗΤΡΑΚΟΠΟΥΛΟΣ 3150033**

**Περιγραφή υλοποίησης**
Το πρόβλημα το προσεγγίσαμε με 2 μοντέλα αλγορίθμων.
Στην πρώτη προσπάθεια χρησιμοποιήσαμε τον booster **XGBRegressor** 
Στην δεύτερη προσπάθεια προσεγγίσαμε το πρόβλημα δημιουργώντας ένα **Νευρωνικό Δίκτυο**


In [59]:
#Importing the libraries 

import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error

In [60]:
# Load data

df_test = pd.read_csv("../input/test.csv")
df_train = pd.read_csv("../input/train.csv")

In [61]:
# One-hot encoding using dummies from panda dataframe
# After one-hot encoding at the test.csv there was not the 4th column so we concatenated it 

dummy_fields = ['yr','season','mnth','weathersit']
for each in dummy_fields:
    dummies = pd.get_dummies(df_train[each], prefix=each, drop_first=False)
    df_train = pd.concat([df_train, dummies], axis=1)
    
fields_to_drop = ['yr','season','mnth','weathersit']
data = df_train.drop(fields_to_drop, axis=1)

dummy_fields2 = ['season','mnth','yr','weathersit']
for each in dummy_fields:
    dummies2 = pd.get_dummies(df_test[each], prefix=each, drop_first=False)
    df_test = pd.concat([df_test, dummies2], axis=1)
    
fields_to_drop = ['yr','season','mnth','weathersit']
data2 = df_test.drop(fields_to_drop, axis=1)

data2['weathersit_4']= df_train['weathersit_4']


In [62]:
# We split the main data

X = data.drop(['casual', 'registered', 'cnt', 'atemp'], axis=1)
y = data['cnt']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
# score function
def rmsle_score(y_true, y_pred):
    for i, y in enumerate(y_pred):
        if y_pred[i] < 0:
            y_pred[i] = 0
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

**XGBRegressor**
Σαν πρώτο μοντέλο χρησιμοποιήσαμε τον αλγόριθμο XGBoost. 
Στην αρχή κάναμε one-hot encoding στα 4 features από πάνω γιατί είδαμε οτί αυτά τα 4 δίνανε το βέλτιστο αποτέσμα.
Κάναμε hyperparameter tuning για να βρούμε ποια νούμερα αποδίδουν καλύτερα στην κάθε μεταβλητή και τα εκχωρήσαμε στον πινακα parameters.
Υλοποιήσαμε cross validation 5 φορές με την βοήθεια του GridSearchCV.
Στη συνέχεια κάναμε fit το μοντέλο μας με τα data που είχαν γινει split.
Η αλήθεια είναι πως ο αλγόριθμος αυτός αν και αποδείχθηκε πιο αποδοτικός απο κάθε άλλο regression αλγόριθμο είχε μεγάλο χρονο εκπαίδευσης της ταξη 2 με 2,5 λεπτά.
Στο καλύτερο αποτέλεσμα που οδηγηθήκαμε ήτανε το 0,326.

In [64]:
rf= XGBRegressor()
parameters = {
 'max_depth': [35],
 'max_features': ['auto'],
 'max_leaf_nodes': [None],
 'min_impurity_decrease': [0.0],
 'min_impurity_split': [None],
 'min_samples_leaf': [1],
 'min_samples_split': [3],
 'min_weight_fraction_leaf': [0.0],
 'n_estimators': [800],
 'random_state': [42],
  }

In [65]:
rf_cv = GridSearchCV(rf, parameters, cv=5, n_jobs=-1)
rf_cv.fit(X_train, y_train)

[14:43:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,...
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [35], 'max_features': ['auto'],
                         'max_leaf_nodes': [None],
                         'min_impurity_decrease': [0.0],
                         'min_impurity_split': [None], 'min_samples_leaf': [1],
                         'min_samples_split': [3],
                  

In [66]:
y_pred= rf_cv.predict(X_test)

print('XGBRegressor RMSLE score:', rmsle_score(y_test, y_pred))

XGBRegressor RMSLE score: 0.32634668462245897


**Νευρωνικά Δίκτυα**
Αυτή την φορά κάνουμε one-hot encoding σε όλα τα categorical features γιατί λειτουργούν καλύτερα τα ΝΝ με περισσότερα data.

In [67]:
# Load data again

df_test = pd.read_csv("../input/test.csv")
df_train = pd.read_csv("../input/train.csv")

In [68]:
# One-hot encoding using dummies from panda dataframe
# After one-hot encoding at the test.csv there was not the 4th column so we concatenated it 

dummy_fields = ['hr','yr','season','mnth','holiday','weekday','workingday','weathersit']
for each in dummy_fields:
    dummies = pd.get_dummies(df_train[each], prefix=each, drop_first=False)
    df_train = pd.concat([df_train, dummies], axis=1)
    
fields_to_drop = ['hr','yr','season','mnth','holiday','weekday','workingday','weathersit']
data = df_train.drop(fields_to_drop, axis=1)

dummy_fields2 = ['hr','season','mnth','yr','holiday','weekday','workingday','weathersit']
for each in dummy_fields:
    dummies2 = pd.get_dummies(df_test[each], prefix=each, drop_first=False)
    df_test = pd.concat([df_test, dummies2], axis=1)
    
fields_to_drop = ['hr','yr','season','mnth','holiday','weekday','workingday','weathersit']
data2 = df_test.drop(fields_to_drop, axis=1)

data2['weathersit_4']= df_train['weathersit_4']



In [69]:
X = data.drop(['casual', 'registered', 'cnt', 'atemp'], axis=1)
y = data['cnt']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Εδώ ορίζουμε το Νευρωνικό μας δίκτυο.**
Στο input layer έχουμε 59 input parameters και αποφασίζουμε να χρησιμοποιήσουμε 64 input nodes.
Σαν activation function επιλέξαμε την regression linear units.
Έπειτα από αρκετές δοκιμές καταλήξαμε οτι βέλτιστο είναι να χρησιμοποιήσουμε 3 hidden layers των 64 nodes έκαστως και ξανα relu για activation function.
Τέλος το ouput layer έχει 1 μεταβλητή και σαν activation το relu ξανά.


In [70]:
# Defining the neural network.

NN_model = Sequential()
# The Input Layer :
NN_model.add(Dense(64, kernel_initializer='uniform',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(64, kernel_initializer='uniform',activation='relu'))
NN_model.add(Dense(64, kernel_initializer='uniform',activation='relu'))
NN_model.add(Dense(64, kernel_initializer='uniform',activation='relu'))


# The Output Layer :
NN_model.add(Dense(1,kernel_initializer='uniform', activation='relu'))


**Optimizer**
Σε αυτό το σημείο χρησιμοποήσαμε τον optimizer Adadelta με τις συγκεκριμένες παραμέτρους για learning rate και decay γιατί λειτούργησαν καλύτερα από RMSprop και Adam.

In [71]:
# Optimizer for the compile function
opt=  keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer=opt, metrics=['mean_absolute_error'],)

Γεμίζουμε το μοντέλο μας με τα split data και επιλέξαμε 40 φορές να τρέξει το ΝΝ  και μέγεθος sample 20 ώστε να αποφύγουμε under και overfitting. Ο αλγόριθμος χρειάζεται μόλις 40 δευτερόλεπτα για να κανει το train και το αποτέλεσμα του ειναι 0,31.

In [72]:
# We fit the neural network with our split data from above
NN_model.fit(X_train, y_train, epochs=40, batch_size=20)

Epoch 1/40
9732/9732 [==============================] - 1s 98us/step - loss: 102.8040 - mean_absolute_error: 102.8040
Epoch 2/40
9732/9732 [==============================] - 1s 77us/step - loss: 57.5021 - mean_absolute_error: 57.5021
Epoch 3/40
9732/9732 [==============================] - 1s 78us/step - loss: 40.0974 - mean_absolute_error: 40.0974
Epoch 4/40
9732/9732 [==============================] - 1s 78us/step - loss: 32.9785 - mean_absolute_error: 32.9785
Epoch 5/40
9732/9732 [==============================] - 1s 78us/step - loss: 30.9636 - mean_absolute_error: 30.9637
Epoch 6/40
9732/9732 [==============================] - 1s 77us/step - loss: 29.9988 - mean_absolute_error: 29.9988
Epoch 7/40
9732/9732 [==============================] - 1s 77us/step - loss: 29.1298 - mean_absolute_error: 29.1298
Epoch 8/40
9732/9732 [==============================] - 1s 78us/step - loss: 28.4580 - mean_absolute_error: 28.4580
Epoch 9/40
9732/9732 [==============================] - 1s 77us/step -

In [73]:
y_pred= NN_model.predict(X_test)
print('Neural Network RMSLE score:', rmsle_score(y_test, y_pred))

Neural Network RMSLE score: 0.3129279019717573


Το μοντέλο του Νευρωνικού δικτύου σε 40 δευτερόλεπτα έκανε train και έχει έτοιμα τα predicted στοιχεία και με σκορ 0,31.

In [74]:
predictions = NN_model.predict(data2.drop(['atemp'], axis=1))

**Τελευταίο Στάδιο**
Φτιάχνουμε τις δύο στήλες Id και Predicted και περνάμε μέσα στο dataframe τα νούμερα που βρήκε το ΝΝ.

In [75]:
submission = pd.DataFrame()
submission['Id'] = range(predictions.shape[0])
submission['Predicted'] = predictions


filename = 'Submission.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Submission.csv


**Παρόλου που στις υποδείξεις για βελτίωση του μοντέλου μας ηταν το να κανουμε predict ξεχωριστα τα casual/registered δεν βοήθησε στην δική μας περίπτωση.**
Δεν υλοποήσαμε κάποια τεχνική dimentional reduction και ίσως να βοηθούσε το μοντέλο μας σε ένα βαθμό. Επίσης χρησιμοποιήσαμε την τεχνική scaling ώστε να αναπαραστίσουμε τα δεδομένα μας σε διαστήμα 0-1 όμως δεν είδαμε καποια βελτίωση ή τελος παντων δεν ηταν ουσιαστική. 